# Bigramas, TF-IDF, PCA

El objetivo de esta sección es tomar como input el pre-procesamiento de la información realizada en los pasos anteriores (interpretación de emojis, stopwords, lematización, etc) y explorar el comportamiento cuando se realiza el análisis por bigrama, TF-IDF y PCA


Aplicaremos:

*   Tf-Idf
*   PCA



In [2]:
!pip install transformers pandas scikit-learn accelerate emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 38.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
import torch
import pandas as pd
import numpy as np
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [4]:
# Verificar si hay una GPU disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Imprimir información sobre los dispositivos CUDA
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


Usando dispositivo: cuda
True
1
Tesla T4


In [7]:
# ejecutar luego de subir archivo excel o tener vinculado Google Drive
import os
import pandas as pd
from google.colab import drive
# mensajes_preprocesados.csv
# columna: sin_stop_words_significado_emoticones

if os.path.isfile("/content/mensajes_trabajo_final1.xlsx"):
  data = pd.read_excel("/content/mensajes_trabajo_final.xlsx")
else:
  # Montar Google Drive
  drive.mount('/content/drive')

  # Cambia el path al archivo que hayas subido a tu Google Drive
  file_path = '/content/drive/MyDrive/UCU-NLP/mensajes_pre_procesados.xlsx'

  # Cargar el archivo Excel
  data = pd.read_excel(file_path)

print("Datos cargados")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Datos cargados


In [8]:
# Seleccionar las columnas relevantes
data = data[['enviado','recibido', 'clase','sin_stop_words_significado_emoticones']]

# Mostrar las primeras filas del archivo
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido  \
0                                               👍🏽🫶🏽   
1                                           Hola....   
2                                       👏🏻👏🏻 gracias   
3                                            Gracias   
4  Hola .. si suele..cuando me...desvelo..o de ta...   

                            clase sin_stop_words_significado_emoticones  
0                  agradecimiento                     aprobación afecto  
1                          saludo                                  hola  
2                  agradecimiento             aplausos aplausos gracias  
3                  agradecimie

In [9]:
# agrupar clases

data.loc[data['clase'] == 'agradecimiento', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'saludo', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'medicación', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'turno', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'queja', 'clase'] = 'otros'
data.loc[data['clase'] == 'pregunta', 'clase'] = 'otros'
data.loc[data['clase'] == 'respuesta a consejo o pregunta', 'clase'] = 'respuesta'

In [10]:
# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

In [11]:
# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

# Filtrar las filas con etiquetas inválidas
data = data[data['clases_cat'] >= 0]

# Verificar que todas las etiquetas estén en el rango correcto
num_labels = len(data['clase'].unique())
print(f"Numero de etiquetas: {num_labels}")
print(f"Etiquetas unicas: {data['clases_cat'].unique()}")
assert data['clases_cat'].min() >= 0 and data['clases_cat'].max() < num_labels

Numero de etiquetas: 9
Etiquetas unicas: [0 8 2 3 7 6 5 1 4]


In [12]:
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido                    clase  \
0                                               👍🏽🫶🏽  agradecimiento o saludo   
1                                           Hola....  agradecimiento o saludo   
2                                       👏🏻👏🏻 gracias  agradecimiento o saludo   
3                                            Gracias  agradecimiento o saludo   
4  Hola .. si suele..cuando me...desvelo..o de ta...                respuesta   

  sin_stop_words_significado_emoticones  clases_cat  
0                     aprobación afecto           0  
1                                  hola           0  
2             

In [13]:
# Reemplazar los valores NaN y vacíos por ''
data['sin_stop_words_significado_emoticones'].fillna('', inplace=True)
data['sin_stop_words_significado_emoticones'] = data['sin_stop_words_significado_emoticones'].apply(lambda x: x.strip() if isinstance(x, str) else x)
data['sin_stop_words_significado_emoticones'].replace('', '', inplace=True)

# Extraer la columna 'sin_stop_words_significado_emoticones'
text_data = data['sin_stop_words_significado_emoticones']

# De la depuración anterior, quedan 7008 mensajes

In [16]:
# Crear una instancia de CountVectorizer
vectorizer = CountVectorizer()

# Ajustar y transformar los datos de texto
X = vectorizer.fit_transform(text_data)

# Convertir la matriz de características a un DataFrame
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [17]:
# Obtener el número total de palabras distintas
total_distinct_words = len(vectorizer.get_feature_names_out())
print("Total de palabras distintas en el corpus:", total_distinct_words)

# Mostrar la matriz de características
print("Matriz de característidas distintas: ")
print(X_df)
print("Matriz de primeras 10 características de los 10 primeros mensajes: ")
print(X_df.iloc[:10, :10])  # Aquí mostramos las primeras 10 filas (comentarios de usuarios) y 10 columnas para una visualización más legible.

Total de palabras distintas en el corpus: 2737
Matriz de característidas distintas: 
      00  04  05  06  08  0800  09  10  100  101  ...  últimos  única  \
0      0   0   0   0   0     0   0   0    0    0  ...        0      0   
1      0   0   0   0   0     0   0   0    0    0  ...        0      0   
2      0   0   0   0   0     0   0   0    0    0  ...        0      0   
3      0   0   0   0   0     0   0   0    0    0  ...        0      0   
4      0   0   0   0   0     0   0   0    0    0  ...        0      0   
...   ..  ..  ..  ..  ..   ...  ..  ..  ...  ...  ...      ...    ...   
7003   0   0   0   0   0     0   0   0    0    0  ...        0      0   
7004   0   0   0   0   0     0   0   0    0    0  ...        0      0   
7005   0   0   0   0   0     0   0   0    0    0  ...        0      0   
7006   0   0   0   0   0     0   0   0    0    0  ...        0      0   
7007   0   0   0   0   0     0   0   0    0    0  ...        0      0   

      únicamente  único  útil  útiles 

Dada la cantidad de palabras que componen nuestro corpus, la matriz de características total no aporta para su visualización. Se realiza filtrado de las 10 primeras columnas de los 10 primeros mensajes, pero el resultado visual tampoco es útil.

Se toman los 100 primeros mensajes y se ordena para cada línea como es la frecuencia de las palabras.


In [21]:
# Limitar el DataFrame a las primeras 100 líneas
X_df_100 = X_df.iloc[:100]

# Contar y ordenar las palabras en cada línea
sorted_word_counts_per_line = []

for index, row in X_df_100.iterrows():
    # Obtener las palabras y sus conteos
    word_counts = row[row > 0].sort_values(ascending=False)
    # Crear una lista de tuplas (palabra, conteo)
    sorted_words = [(word, count) for word, count in zip(word_counts.index, word_counts.values)]
    sorted_word_counts_per_line.append(sorted_words)

# Mostrar las palabras ordenadas por línea
for i, sorted_words in enumerate(sorted_word_counts_per_line):
    print(f"Documento {i + 1}: {sorted_words}")

Documento 1: [('afecto', 1), ('aprobación', 1)]
Documento 2: [('hola', 1)]
Documento 3: [('aplausos', 2), ('gracias', 1)]
Documento 4: [('gracias', 1)]
Documento 5: [('cuando', 1), ('desvelo', 1), ('hola', 1), ('si', 1), ('suele', 1), ('tarde', 1)]
Documento 6: [('gracias', 1), ('muchas', 1)]
Documento 7: [('homa', 1)]
Documento 8: [('gracias', 1), ('muchas', 1)]
Documento 9: [('consejos', 1), ('cuenta', 1), ('gracias', 1), ('muchas', 1), ('tengo', 1)]
Documento 10: [('complicidad', 1)]
Documento 11: [('gracias', 1), ('información', 1), ('no', 1), ('sabía', 1)]
Documento 12: [('gracias', 1)]
Documento 13: [('bueno', 1), ('gracias', 1)]
Documento 14: [('felicidad', 2), ('bueno', 1), ('gracias', 1)]
Documento 15: [('aprobación', 1), ('ok', 1)]
Documento 16: [('gracias', 1)]
Documento 17: [('si', 1)]
Documento 18: [('camino', 1), ('hay', 1), ('moverse', 1), ('si', 1)]
Documento 19: [('acuerdo', 1), ('consejo', 1), ('gracias', 1), ('habitual', 1), ('rutina', 1), ('saludo', 1), ('totalmente


**BI-GRAMAS**

In [24]:
# Crear una instancia de CountVectorizer con ngram_range para bi-gramas
vectorizer_bigrama = CountVectorizer(ngram_range=(2, 2))

# Ajustar y transformar los datos de texto
X_bigrama = vectorizer_bigrama.fit_transform(text_data)

# Convertir la matriz de características a un DataFrame
X_df_bigrama = pd.DataFrame(X_bigrama.toarray(), columns=vectorizer_bigrama.get_feature_names_out())



In [ ]:
# Mostrar la matriz de características
print(X_df_bigrama)
print(X_df_bigrama.iloc[:10, :10])  # Mostramos las primeras 10 filas y 10 columnas para una visualización más legible.

In [44]:
# Configurar Pandas para mostrar todas las filas
pd.set_option('display.max_rows', None)

In [48]:
# Sumar las ocurrencias de cada bi-grama en todos los documentos
bigram_counts = X_df_bigrama.sum(axis=0)

# Filtrar los bi-gramas cuya cantidad es mayor a 1
bigram_counts_filtered = bigram_counts[bigram_counts > 5]

print(bigram_counts_filtered)

# Ordenar los bi-gramas por sus conteos en orden descendente
bigram_counts_sorted = bigram_counts_filtered.sort_values(ascending=False)

# Seleccionar los 60 bi-gramas más frecuentes
top_60_bigrams = bigram_counts_sorted.head(7468)

# Mostrar los 20 bi-gramas más frecuentes
print(top_60_bigrams)

20 horas                     7
30 minutos                   7
60 minutos                   7
768 deja                    14
87 igshid                   14
actualmente no              14
afecto afecto               15
afecto aprobación            7
aplausos aplausos           52
aprobación afecto            7
aprobación aprobación      123
aprobación felicidad         6
aprobación gracias          19
así ves                     12
atendemos días               7
atendemos lunes              7
ayudarte felicidad           9
año pasado                  12
bajar peso                  23
beso afecto                  7
beso beso                   45
beso nombre                 12
bien gracias                45
breve contigo               12
buen día                   271
buenas tarde                19
buenas tardes              118
bueno día                    7
bueno gracias               96
bueno muchas                20
buenos dias                 17
buenos día                  11
buenos d

In [33]:
# Cantidad total de bi-gramas detectados
print("Cantidad total de bigramas detectados: ")
print(len(bigram_counts_sorted))

Cantidad total de bigramas detectados: 
7468



**TF-IDF**

In [35]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)

In [36]:
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(X.toarray(), columns=feature_names)
print(tfidf_df.head(20))  # Mostrar los primeros 20 documentos (textos) con sus TF-IDF

     00   04   05   06   08  0800   09   10  100  101  ...  últimos  única  \
0   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
1   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
2   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
3   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
4   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
5   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
6   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
7   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
8   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
9   0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
10  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      0.0    0.0   
11  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...      

Listamos las palabras mas relevantes para cada línea tomando los 20 primeros mensajes.

In [37]:
# Listar las palabras más relevantes de cada línea
for index, row in tfidf_df.iterrows():
    if index >= 20:  # Limitar a los primeros 20 mensajes
        break
    # Obtener los índices de las palabras con mayor TF-IDF
    top_indices = row.argsort()[::-1][:5]  # Obtener los 5 más altos
    top_words = [(feature_names[i], row[i]) for i in top_indices if row[i] > 0]  # Filtrar palabras con valor > 0
    print(f'Documento {index + 1}:', top_words)

Documento 1: [('afecto', 0.823781038334871), ('aprobación', 0.5669081062041023)]
Documento 2: [('hola', 1.0)]
Documento 3: [('aplausos', 0.9848180435437227), ('gracias', 0.17358980704728674)]
Documento 4: [('gracias', 1.0)]
Documento 5: [('desvelo', 0.564429493625928), ('suele', 0.564429493625928), ('tarde', 0.3955750744837958), ('cuando', 0.3917233945351342), ('hola', 0.19508386350892104)]
Documento 6: [('muchas', 0.8583240043798179), ('gracias', 0.5131080816995521)]
Documento 7: [('homa', 1.0)]
Documento 8: [('muchas', 0.8583240043798179), ('gracias', 0.5131080816995521)]
Documento 9: [('cuenta', 0.6402570655091071), ('consejos', 0.4988311901431382), ('tengo', 0.41433367514610775), ('muchas', 0.3534439634098043), ('gracias', 0.211289621550931)]
Documento 10: [('complicidad', 1.0)]
Documento 11: [('sabía', 0.635461858821152), ('información', 0.622919871667618), ('no', 0.3781643386190519), ('gracias', 0.25524653271294384)]
Documento 12: [('gracias', 1.0)]
Documento 13: [('bueno', 0.883

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Suponiendo que el DataFrame text_data tiene una columna llamada 'mensaje' que contiene los textos
corpus = data['sin_stop_words_significado_emoticones'].tolist()  # Extraer los mensajes como lista

# Calcular TF-IDF para todo el corpus
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Convertir a DataFrame para un análisis más fácil
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Calcular la media de TF-IDF para cada palabra en el corpus
mean_tfidf = tfidf_df.mean(axis=0)

# Obtener las palabras más relevantes (mayor TF-IDF medio)
top_indices = mean_tfidf.argsort()[::-1][:50]  # Top 50 palabras
top_words = [(vectorizer.get_feature_names_out()[i], mean_tfidf[i]) for i in top_indices]

print('Palabras más relevantes del corpus:', top_words)


Palabras más relevantes del corpus: [('si', 0.18480547426289848), ('quiero', 0.14828149114707664), ('recibirlos', 0.14570658996781696), ('gracias', 0.14032860467186695), ('ok', 0.07807025254902526), ('aprobación', 0.058472436023531915), ('hola', 0.048633828821304584), ('no', 0.03941531381726468), ('muchas', 0.03444622404403997), ('bueno', 0.0283134923256725), ('día', 0.018610912485680163), ('buen', 0.0168947916412516), ('bien', 0.012308274412550928), ('tengo', 0.009239902697340707), ('felicidad', 0.009167369431504405), ('buenas', 0.008650247147665732), ('nombre', 0.008426179838694375), ('buenos', 0.00815083183792824), ('afecto', 0.008017015756589689), ('días', 0.007261959707375985), ('tardes', 0.0070686130550422464), ('consejos', 0.006864328161317478), ('información', 0.006360474702830439), ('beso', 0.0059535635358625785), ('sabía', 0.005948996563821993), ('consejo', 0.005799066516842392), ('turno', 0.005356295933515087), ('ya', 0.005246709252292988), ('sabia', 0.005154624992757991), (


Continuamos con el análisis, aplicamos PCA para la reducción de dimensionalidad y de esta manera quedarnos con las 50 palabras mas representativas.

In [58]:
# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=50)
principal_components = pca.fit_transform(tfidf_df)

# Convertir los componentes principales a un DataFrame para visualización
pca_df = pd.DataFrame(principal_components)
# Mostrar los componentes principales
print(pca_df.head(50))


          0         1         2         3         4         5         6   \
0  -0.244238 -0.380353 -0.204868  0.359428  0.026911  0.000031  0.002991   
1  -0.240467 -0.232819 -0.146956 -0.395570 -0.018003  0.793838  0.025626   
2  -0.243661 -0.011213 -0.075535 -0.062047 -0.046567 -0.060124  0.000269   
3  -0.487200  0.717007 -0.152867  0.163067 -0.005572  0.000539 -0.075024   
4  -0.136921 -0.158903 -0.075368 -0.178435  0.048842  0.090228  0.000134   
5  -0.396229  0.398628 -0.145910  0.031557 -0.045248  0.033804  0.029214   
6  -0.188000 -0.155549 -0.055226 -0.106189 -0.052004 -0.063725  0.014849   
7  -0.396229  0.398628 -0.145910  0.031557 -0.045248  0.033804  0.029214   
8  -0.283180  0.068644 -0.101423 -0.066503 -0.059853 -0.034184  0.014448   
9  -0.189046 -0.158991 -0.057197 -0.103270 -0.052141 -0.064764  0.015794   
10 -0.283206  0.024860 -0.113006 -0.107145 -0.177388 -0.188994 -0.227037   
11 -0.487200  0.717007 -0.152867  0.163067 -0.005572  0.000539 -0.075024   
12 -0.361786

In [59]:
# Identificar las palabras más importantes en los componentes principales
# Para esto, necesitamos obtener las cargas de los componentes
components = pca.components_

# Crear un DataFrame con las cargas de los componentes
components_df = pd.DataFrame(components, columns=feature_names)

In [60]:
print(components.shape)

(50, 2737)


In [61]:
# Para cada mensaje, mostrar las palabras con las mayores cargas absolutas
top_n = 20
for i in range(components.shape[0]):
    print(f"Componente Principal {i + 1}:")
    sorted_components = components_df.iloc[i].abs().sort_values(ascending=False)
    print(sorted_components.head(top_n))

Componente Principal 1:
recibirlos     0.552460
quiero         0.545682
si             0.513729
gracias        0.299340
ok             0.156251
aprobación     0.085383
muchas         0.063816
hola           0.052607
bueno          0.037904
no             0.025726
día            0.021889
buen           0.020268
bien           0.015052
felicidad      0.013433
buenos         0.009858
consejos       0.009767
afecto         0.009679
información    0.009665
nombre         0.009451
tengo          0.009257
Name: 0, dtype: float64
Componente Principal 2:
gracias       0.872257
aprobación    0.380264
recibirlos    0.151526
quiero        0.143627
muchas        0.123866
si            0.113852
no            0.100103
hola          0.077568
ok            0.043128
día           0.026476
buen          0.022898
tengo         0.017612
buenas        0.014939
nombre        0.014586
tardes        0.012016
afecto        0.011566
sabía         0.011561
hace          0.011026
turno         0.010966
sabia      

El resultado anterior lo muestra abierto por cada mensaje procesado (línea de comentario del dataframe), en nuestro caso indicamos las 50 primeras y por ese motivo solo vemos 50 componentes procesados.
Para cada línea muestra las palabras con su respectivo peso/carga

Ahora sumamos las puntuaciones de TF-IDF para cada palabra de todo el corpus y obtener así una visión general de las 50 palabras con mayor puntaje.

In [64]:
# Calcular el peso total de cada palabra en todos los documentos basado en los componentes principales del PCA
total_weight_per_word = components_df.abs().sum(axis=0)

# Ordenar las palabras por su peso total descendente
sorted_total_weight = total_weight_per_word.sort_values(ascending=False)

# Mostrar las 50 palabras con mayor peso total
top_50_total_weight_words = sorted_total_weight.head(50)
print(top_50_total_weight_words)


tengo          3.869096
recibirlos     3.853293
quiero         3.822941
hace           3.495742
ya             3.401840
turno          3.344127
no             3.334598
número         3.285361
bendiciones    3.266684
año            3.102251
meses          3.065491
muy            3.028364
información    2.982189
interesa       2.938578
sabía          2.917084
nada           2.910110
recibirlo      2.882392
nombre         2.880821
aplausos       2.863336
chispas        2.787079
recibir        2.778426
consejo        2.778100
hice           2.774132
dias           2.765681
felicidad      2.666330
aprobación     2.626729
si             2.588408
sabia          2.499307
consejos       2.495912
beso           2.473376
afecto         2.472687
mensaje        2.452480
baja           2.445921
bueno          2.442605
hago           2.431471
días           2.430537
gracias        2.405948
cómo           2.399837
30             2.379223
60             2.379188
hola           2.359601
minutos        2